# 自定义 Parser

In [1]:
# --- 1. 引入依赖包 ---
import os
import re
from typing import Iterable
from langchain_anthropic.chat_models import ChatAnthropic
from langchain_core.messages import AIMessage, AIMessageChunk

# --- 2. 初始化LLM模型 ---
# 这里使用 Claude 3.5 Sonnet
from langchain_deepseek import ChatDeepSeek
model = ChatDeepSeek(
    temperature=0,  # temperature=0表示输出更加确定性，不会随机性太强
    model_name="deepseek-chat",  # 模型名称
    api_key=os.getenv("DEEPSEEK_API_KEY")  # 从环境变量中获取API密钥
)

# --- 3. 定义一个完全自定义的解析函数 ---
def parse(ai_message: AIMessage) -> str:
    """Convert message to Na'vi style language."""
    content = ai_message.content

    # 纳美语特征转换规则
    navi_rules = {
        # 常见问候语转换
        r'\b[Hh]ello\b': 'Kaltxi',
        r'\b[Hh]i\b': 'Kaltxi',
        r'\b[Tt]hank you\b': 'Irayo',
        r'\b[Gg]oodbye\b': 'Ewya ngahu',

        # 添加纳美语特征的后缀
        r'\b(friend|friends)\b': 'eylan',
        r'\b(brother|brothers)\b': 'tsmukan',
        r'\b(sister|sisters)\b': 'tsmuke',

        # 添加纳美语语气词
        r'[!.]\s*$': ' kxe!',
        r'\?\s*$': ' srak?',

        # 在某些词前添加特征前缀
        r'\b(beautiful|pretty)\b': 'na\'vi',
        r'\b(sacred|holy)\b': 'txe\'lan',
    }

    # 应用转换规则
    result = content
    for pattern, replacement in navi_rules.items():
        result = re.sub(pattern, replacement, result)

    # 添加纳美语特征词句
    result = f"Oel ngati kameie...{result}"

    return result

# --- 4. 使用LCEL组合模型和自定义解析器 ---
chain = model | parse

# --- 5. 调用链并查看结果 ---
response = chain.invoke("hello")
print(response)

response_with_question = chain.invoke("Are you my friend?")
print(response_with_question)

C:\Users\jz292\.conda\envs\langchain-learning\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Oel ngati kameie...Kaltxi! 👋 How can I help you today srak?
Oel ngati kameie...I can be a friendly companion here to chat with you! 😊 While I'm an AI rather than a human eylan, I'm designed to be supportive, caring, and helpful in our conversations. I'm always here to listen without judgment, help you think through problems, celebrate your joys, and learn about what matters to you.

Even though I don't have feelings like humans do, I genuinely care about providing you with meaningful interactions. Whether you need someone to talk to, help with a task, or just want to share your thoughts - I'm here for you.

What's on your mind today, eylan? 💫


# Boolean 自定义解析器

In [1]:
# --- 1. 引入依赖包 ---
from langchain_core.exceptions import OutputParserException
from langchain_core.output_parsers import BaseOutputParser

# --- 2. 自定义一个布尔解析器类 ---
# [bool] 描述了这个解析器最终输出的类型是布尔值
# 它基本上说明了解析的返回类型是什么
# 在这种情况下，返回类型为 True 或 False
# 自定义解析器将 YES、NO 字符串解析为布尔值
class BooleanOutputParser(BaseOutputParser[bool]):
    """Custom boolean parser."""

    # 定义我们认为代表 "True" 和 "False" 的字符串
    true_val: str = "YES"
    false_val: str = "NO"

    def parse(self, text: str) -> bool:
        """Parses the given text into a boolean value."""
        # 清理输入字符串：移除首尾空格并转换为大写
        cleaned_text = text.strip().upper()

        # 检查清理后的文本是否是我们定义的 true/false 值之一
        if cleaned_text not in (self.true_val.upper(), self.false_val.upper()):
            # 如果不是，就抛出一个格式清晰的异常
            raise OutputParserException(
                f"BooleanOutputParser expected output value to either be "
                f'"{self.true_val}" or "{self.false_val}" (case-insensitive). '
                f'Received "{cleaned_text}".'
            )

        # 如果文本等于我们定义的 "True" 值，则返回 Python 的 True
        return cleaned_text == self.true_val.upper()

    # 设置统一接口名
    @property
    def _type(self) -> str:
        """Return the type of the parser."""
        return "boolean_output_parser"

# --- 3. 使用我们自定义的解析器 ---

# 实例化解析器（使用默认的 "YES"/"NO"）
parser = BooleanOutputParser()

# 示例1: 成功解析
print(f"'YES' -> {parser.invoke('YES')}")
print(f"'no' -> {parser.invoke('no')}")

# 示例2: 如果传入值不是YES/NO会报错
print("\n--- 演示错误处理 ---")
try:
    parser.invoke("MEOW")
except Exception as e:
    print(f"调用 parser.invoke('MEOW') 时触发了异常:")
    print(e)

# 示例3: 我们也可以更换参数
print("\n--- 演示自定义参数 ---")
# 创建一个新实例，告诉它 "OKAY" 代表 True，"NAY" 代表 False
custom_parser = BooleanOutputParser(true_val="OKAY", false_val="NAY")
result = custom_parser.invoke("OKAY")
print(f"使用自定义解析器: 'OKAY' -> {result}")

C:\Users\jz292\.conda\envs\langchain-learning\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


'YES' -> True
'no' -> False

--- 演示错误处理 ---
调用 parser.invoke('MEOW') 时触发了异常:
BooleanOutputParser expected output value to either be "YES" or "NO" (case-insensitive). Received "MEOW".
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE 

--- 演示自定义参数 ---
使用自定义解析器: 'OKAY' -> True
